In [154]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
import numpy as np
import re

In [155]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [156]:
dossier='/content/drive/MyDrive/'
df=pd.read_csv(dossier+'text_preprocessed_boubacar.csv')
df=df.astype(str).sample(df.shape[0]//100)
df_questions=df["Content"]
df_answers=df["Answers"]

In [157]:
df

,Unnamed: 0,Title,Content,Answers
163281,163281,hello world java : understanding concept versu...,trying learn java hello world program already ...,as you know python uses duck typing it does no...
291104,291104,slice list ordered chunks,dictionary like : item_count_per_section = { 1...,dict comprehension of itertools isliced iter o...
183442,183442,smtplib sends blank message message contain ce...,"current script allows send emails fine , chara...",i do not think it is about the colon but it is...
415947,415947,count boolean grouped spark data frame,want count many records true column grouped sp...,probably the simplest solution is a plain cast...
226655,226655,intellij idea 12 : get pep8 fly,seems already supported pycharm 2.7 get intell...,available in idea with the new python plug in
...,...,...,...,...
80146,80146,interactively validating entry widget content ...,recommended technique interactively validating...,the correct answer is use the validatecommand ...
18340,18340,host use making udp socket python ?,want ro receive data sent udp packet vpn . wro...,use sock bind
86232,86232,comparing two dictionaries python,"two dictionaries , simplification , take two :...",what you want to do is simply x y what you do ...
223384,223384,python pil struggles uncompressed 16-bit tiff ...,system mac os x v10.8.2 . several 2560x500 unc...,try pillow the friendly pil fork they have som...


In [158]:
df.head()


,Unnamed: 0,Title,Content,Answers
163281,163281,hello world java : understanding concept versu...,trying learn java hello world program already ...,as you know python uses duck typing it does no...
291104,291104,slice list ordered chunks,dictionary like : item_count_per_section = { 1...,dict comprehension of itertools isliced iter o...
183442,183442,smtplib sends blank message message contain ce...,"current script allows send emails fine , chara...",i do not think it is about the colon but it is...
415947,415947,count boolean grouped spark data frame,want count many records true column grouped sp...,probably the simplest solution is a plain cast...
226655,226655,intellij idea 12 : get pep8 fly,seems already supported pycharm 2.7 get intell...,available in idea with the new python plug in


In [159]:
questions_train, questions_test, answers_train, answers_test = train_test_split(df_questions,df_answers, test_size=0.2, random_state=42)

In [160]:
from keras.preprocessing.text import Tokenizer

from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

# Créer un tokenizer pour convertir les phrases en séquences de mots
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions_train)

# Convertir les phrases en séquences de mots
questions_train = tokenizer.texts_to_sequences(questions_train)
questions_test = tokenizer.texts_to_sequences(questions_test)


# Créer un tokenizer pour convertir les phrases en séquences de mots
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(answers_train)

# Convertir les phrases en séquences de mots
answers_train = tokenizer.texts_to_sequences(answers_train)
answers_test = tokenizer.texts_to_sequences(answers_test)

max_length_questions = max([len(s) for s in questions_train])
max_length_answers = max([len(s) for s in answers_train])
max_length=max([max_length_questions, max_length_answers])
# Pad les séquences pour qu'elles aient toutes la même longueur

questions_train = pad_sequences(questions_train, maxlen=max_length)
questions_test = pad_sequences(questions_test, maxlen=max_length)


# Pad les séquences pour qu'elles aient toutes la même longueur

answers_train = pad_sequences(answers_train, maxlen=max_length)
answers_test = pad_sequences(answers_test, maxlen=max_length)


# Créer un encoder-decoder avec un LSTM
encoder_inputs = Input(shape=(max_length,1))
encoder = Embedding(input_dim=max_length, output_dim=64, input_length=max_length)(encoder_inputs)
encoder = LSTM(64, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_length,))
decoder_embedding = Embedding(input_dim=max_length, output_dim=64, input_length=max_length)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(max_length, activation='softmax')
outputs = decoder_dense(decoder_outputs)


In [168]:

# Compiler le modèle
model = Model([encoder_inputs, decoder_inputs], [outputs])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entraîner le modèle
model.fit([questions_train, questions_train], answers_train, epochs=8, batch_size=8, validation_split=0.2, steps_per_epoch=1, validation_steps=1)


Epoch 1/8
1/1 [==============================] - 3s 3s/step - loss: nan - accuracy: 0.9623 - val_loss: nan - val_accuracy: 0.9791
Epoch 2/8
1/1 [==============================] - 0s 444ms/step - loss: nan - accuracy: 0.9794 - val_loss: nan - val_accuracy: 0.9791
Epoch 3/8
1/1 [==============================] - 1s 579ms/step - loss: nan - accuracy: 0.9730 - val_loss: nan - val_accuracy: 0.9791
Epoch 4/8
1/1 [==============================] - 0s 439ms/step - loss: nan - accuracy: 0.9800 - val_loss: nan - val_accuracy: 0.9791
Epoch 5/8
1/1 [==============================] - 0s 436ms/step - loss: nan - accuracy: 0.9813 - val_loss: nan - val_accuracy: 0.9791
Epoch 6/8
1/1 [==============================] - 0s 436ms/step - loss: nan - accuracy: 0.9794 - val_loss: nan - val_accuracy: 0.9791
Epoch 7/8
1/1 [==============================] - 1s 593ms/step - loss: nan - accuracy: 0.9705 - val_loss: nan - val_accuracy: 0.9791
Epoch 8/8
1/1 [==============================] - 1s 581ms/step - loss: n

In [169]:
# Évaluer le modèle sur les données de test
loss, acc = model.evaluate([questions_test, questions_test], answers_test, verbose=0, batch_size=8)
print("Test accuracy:", acc)

Test accuracy: 0.9728500843048096


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')



In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
from nltk.stem import *
def stem_traincorpus(data):
    stemmer = PorterStemmer()
    out_data=""
    for words in data:
        out_data+= stemmer.stem(words)
    return out_data
def remove_stopwords(data):
  text=data.lower()
  text_tokens = word_tokenize(text)
  tokens_without_sw = [word for word in text_tokens if not word in stop_words]
  return (" ").join(tokens_without_sw)

In [163]:
import keras
#def prediction
text=df.iloc[-2]["Content"]
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([text])
sequence = tokenizer.texts_to_sequences([text])
sequence = pad_sequences(sequence, maxlen = max_length)
predict = model.predict([sequence,sequence]) 
sequence_returned=np.argmax(predict,axis=1)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_sequences(sequence_returned)
new_text=tokenizer.sequences_to_texts(sequence_returned)
print(new_text)
#tableau=np.array([questions_train, answers_train])
#tableau.reshape(1,tableau.shape[1],tableau.shape[2])

1/1 [==============================] - 1s 831ms/step
['']


In [164]:
[questions_train, questions_train]

[array([[    0,     0,     0, ...,   103, 15893,    93],
        [    0,     0,     0, ...,   430,  1183,  3727],
        [    0,     0,     0, ...,   115,     2, 11113],
        ...,
        [    0,     0,     0, ...,  1665,  1389,    32],
        [    0,     0,     0, ...,  1303, 14675,   573],
        [    0,     0,     0, ...,    12,   211,   173]], dtype=int32),
 array([[    0,     0,     0, ...,   103, 15893,    93],
        [    0,     0,     0, ...,   430,  1183,  3727],
        [    0,     0,     0, ...,   115,     2, 11113],
        ...,
        [    0,     0,     0, ...,  1665,  1389,    32],
        [    0,     0,     0, ...,  1303, 14675,   573],
        [    0,     0,     0, ...,    12,   211,   173]], dtype=int32)]